## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sns.set_style("whitegrid", {'grid.linestyle': '--'})
sns.set_style({'font.family':'serif', 'font.serif':'Computer Modern'})
sns.set_context(font_scale=2, rc={"font.size":10,"axes.titlesize":20,"axes.labelsize":15})

In [3]:
from collections import defaultdict

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [6]:
df_train = pd.read_csv("C:/Users/Utente/OneDrive/Desktop/UNIPI - DS for Business Informatics/Data Mining_2/Progetto/df_train_processed.csv")
df_test = pd.read_csv("C:/Users/Utente/OneDrive/Desktop/UNIPI - DS for Business Informatics/Data Mining_2/Progetto/df_test_processed.csv")

In [7]:
from sklearn.svm import LinearSVC

In [8]:
from sklearn.model_selection import RandomizedSearchCV

In [9]:
from sklearn.svm import SVC

In [10]:
pd.set_option('display.max_columns', None)

In [11]:
df_train.head()

,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,frame_count,sum,mean,std,min,max,q01,q05,q25,q50,q75,q95,q99,kur,skew,lag1_sum,lag1_mean,lag1_std,lag1_min,lag1_max,lag1_q01,lag1_q05,lag1_q25,lag1_q75,lag1_q95,lag1_q99,lag1_kur,lag1_skew,zc_sum,zc_mean,zc_std,zc_q95,zc_kur,zc_skew,mfcc_sum,mfcc_mean,mfcc_std,mfcc_min,mfcc_max,mfcc_q01,mfcc_q05,mfcc_q25,mfcc_q50,mfcc_q75,mfcc_q95,mfcc_q99,mfcc_kur,mfcc_skew,sc_sum,sc_mean,sc_std,sc_min,sc_max,sc_q01,sc_q05,sc_q25,sc_q50,sc_q75,sc_q95,sc_q99,sc_kur,sc_skew,stft_sum,stft_mean,stft_std,stft_min,stft_q01,stft_q05,stft_q25,stft_q50,stft_q75,stft_kur,stft_skew,length_w1,sum_w1,mean_w1,std_w1,min_w1,max_w1,q01_w1,q05_w1,q25_w1,q50_w1,q75_w1,q95_w1,q99_w1,kur_w1,skew_w1,lag1_sum_w1,lag1_mean_w1,lag1_std_w1,lag1_min_w1,lag1_max_w1,lag1_q01_w1,lag1_q05_w1,lag1_q25_w1,lag1_q75_w1,lag1_q95_w1,lag1_q99_w1,lag1_kur_w1,lag1_skew_w1,zc_sum_w1,zc_mean_w1,zc_std_w1,zc_q75_w1,zc_q95_w1,zc_q99_w1,zc_kur_w1,zc_skew_w1,mfcc_sum_w1,mfcc_mean_w1,mfcc_std_w1,mfcc_min_w1,mfcc_max_w1,mfcc_q01_w1,mfcc_q05_w1,mfcc_q25_w1,mfcc_q50_w1,mfcc_q75_w1,mfcc_q95_w1,mfcc_q99_w1,mfcc_kur_w1,mfcc_skew_w1,sc_sum_w1,sc_mean_w1,sc_std_w1,sc_min_w1,sc_max_w1,sc_q01_w1,sc_q05_w1,sc_q25_w1,sc_q50_w1,sc_q75_w1,sc_q95_w1,sc_q99_w1,sc_kur_w1,sc_skew_w1,stft_sum_w1,stft_mean_w1,stft_std_w1,stft_min_w1,stft_max_w1,stft_q01_w1,stft_q05_w1,stft_q25_w1,stft_q50_w1,stft_q75_w1,stft_q95_w1,stft_q99_w1,stft_kur_w1,stft_skew_w1,length_w2,sum_w2,mean_w2,std_w2,min_w2,max_w2,q01_w2,q05_w2,q25_w2,q50_w2,q75_w2,q95_w2,q99_w2,kur_w2,skew_w2,lag1_sum_w2,lag1_mean_w2,lag1_std_w2,lag1_min_w2,lag1_max_w2,lag1_q01_w2,lag1_q05_w2,lag1_q25_w2,lag1_q50_w2,lag1_q75_w2,lag1_q95_w2,lag1_q99_w2,lag1_kur_w2,lag1_skew_w2,zc_sum_w2,zc_mean_w2,zc_std_w2,zc_q75_w2,zc_q95_w2,zc_kur_w2,zc_skew_w2,mfcc_sum_w2,mfcc_mean_w2,mfcc_std_w2,mfcc_min_w2,mfcc_max_w2,mfcc_q01_w2,mfcc_q05_w2,mfcc_q25_w2,mfcc_q50_w2,mfcc_q75_w2,mfcc_q95_w2,mfcc_q99_w2,mfcc_kur_w2,mfcc_skew_w2,sc_sum_w2,sc_mean_w2,sc_std_w2,sc_min_w2,sc_max_w2,sc_q01_w2,sc_q05_w2,sc_q25_w2,sc_q50_w2,sc_q75_w2,sc_q95_w2,sc_q99_w2,sc_kur_w2,sc_skew_w2,stft_sum_w2,stft_mean_w2,stft_std_w2,stft_min_w2,stft_q01_w2,stft_q05_w2,stft_q25_w2,stft_q50_w2,stft_q75_w2,stft_kur_w2,stft_skew_w2,length_w3,sum_w3,mean_w3,std_w3,min_w3,max_w3,q01_w3,q05_w3,q25_w3,q50_w3,q75_w3,q95_w3,q99_w3,kur_w3,skew_w3,lag1_sum_w3,lag1_mean_w3,lag1_std_w3,lag1_min_w3,lag1_max_w3,lag1_q01_w3,lag1_q05_w3,lag1_q25_w3,lag1_q50_w3,lag1_q75_w3,lag1_q95_w3,lag1_q99_w3,lag1_kur_w3,lag1_skew_w3,zc_sum_w3,zc_mean_w3,zc_std_w3,zc_q95_w3,zc_kur_w3,zc_skew_w3,mfcc_sum_w3,mfcc_mean_w3,mfcc_std_w3,mfcc_min_w3,mfcc_max_w3,mfcc_q01_w3,mfcc_q05_w3,mfcc_q25_w3,mfcc_q50_w3,mfcc_q75_w3,mfcc_q95_w3,mfcc_q99_w3,mfcc_kur_w3,mfcc_skew_w3,sc_sum_w3,sc_mean_w3,sc_std_w3,sc_min_w3,sc_max_w3,sc_q01_w3,sc_q05_w3,sc_q25_w3,sc_q50_w3,sc_q75_w3,sc_q95_w3,sc_q99_w3,sc_kur_w3,sc_skew_w3,stft_sum_w3,stft_mean_w3,stft_std_w3,stft_min_w3,stft_q01_w3,stft_q05_w3,stft_q25_w3,stft_q50_w3,stft_q75_w3,stft_kur_w3,stft_skew_w3,length_w4,sum_w4,mean_w4,std_w4,min_w4,max_w4,q01_w4,q05_w4,q25_w4,q50_w4,q75_w4,q95_w4,q99_w4,kur_w4,skew_w4,lag1_sum_w4,lag1_mean_w4,lag1_std_w4,lag1_min_w4,lag1_max_w4,lag1_q01_w4,lag1_q05_w4,lag1_q25_w4,lag1_q75_w4,lag1_q95_w4,lag1_q99_w4,lag1_kur_w4,lag1_skew_w4,zc_sum_w4,zc_mean_w4,zc_std_w4,zc_q75_w4,zc_q95_w4,zc_q99_w4,zc_kur_w4,zc_skew_w4,mfcc_sum_w4,mfcc_mean_w4,mfcc_std_w4,mfcc_min_w4,mfcc_max_w4,mfcc_q01_w4,mfcc_q05_w4,mfcc_q25_w4,mfcc_q50_w4,mfcc_q75_w4,mfcc_q95_w4,mfcc_q99_w4,mfcc_kur_w4,mfcc_skew_w4,sc_sum_w4,sc_mean_w4,sc_std_w4,sc_min_w4,sc_max_w4,sc_q01_w4,sc_q05_w4,sc_q25_w4,sc_q50_w4,sc_q75_w4,sc_q95_w4,sc_q99_w4,sc_kur_w4,sc_skew_w4,stft_sum_w4,stft_mean_w4,stft_std_w4,stft_min_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4
0,speech,neutral,normal,Kids are talking by the door,1st,1,M,158558,0.145081,9.150000e-07,0.004001,-0.038422,0.040588,-0.012586,-0.005890,

## Standardization

### Train

In [11]:
df_train_num = df_train.select_dtypes(include="number").drop("actor", axis=1)
df_train_cat = df_train.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_train_num_scaled = pd.DataFrame(scaler.fit_transform(df_train_num), columns=df_train_num.columns)
df_train_scaled_s = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

### Test

In [12]:
df_test_num = df_test.select_dtypes(include="number").drop("actor", axis=1)
df_test_cat = df_test.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_test_num_scaled = pd.DataFrame(scaler.fit_transform(df_test_num), columns=df_test_num.columns)
df_test_scaled_s = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

# Linear SVM

## Variabile Target --> Sex

### One-hot encoding

#### Train 

In [21]:
df_train_scaled_s['sex'] = pd.factorize(df_train_scaled_s['sex'])[0]
df_train_s = pd.get_dummies(df_train_scaled_s)

#### Test

In [22]:
df_test_scaled_s['sex'] = pd.factorize(df_test_scaled_s['sex'])[0]
df_test_s = pd.get_dummies(df_test_scaled_s)

### Modello

In [53]:
X_train = df_train_s.drop(['sex'], axis=1)
y_train = df_train_s['sex']
X_test = df_test_s.drop(['sex'], axis=1)
y_test = df_test_s['sex']

C_range = np.logspace(-4, 4, 100) # genera 100 valori logaritmici distribuiti uniformemente nell'intervallo [0.001, 1000].
param_grid = {'C': C_range}

clf = LinearSVC(dual=False)
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=100, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) # addestro il modello sul train

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred))

print("Best C value:", random_search.best_params_['C'])

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       312
           1       1.00      0.70      0.82       312

    accuracy                           0.85       624
   macro avg       0.88      0.85      0.84       624
weighted avg       0.88      0.85      0.84       624

Best C value: 0.026560877829466867
CPU times: total: 3.3 s
Wall time: 24.3 s


## Variabile target --> Vocal_channel

### One-hot encoding

In [17]:
df_test_scaled_v = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [18]:
df_train_scaled_v = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

#### Train 

In [19]:
df_train_scaled_v['vocal_channel'] = pd.factorize(df_train_scaled_v['vocal_channel'])[0]
df_train_v = pd.get_dummies(df_train_scaled_v)

#### Test

In [20]:
df_test_scaled_v['vocal_channel'] = pd.factorize(df_test_scaled_v['vocal_channel'])[0]
df_test_v = pd.get_dummies(df_test_scaled_v)

In [21]:
df_train_v.head()

,frame_count,sum,mean,std,min,max,q01,q05,q25,q50,q75,q95,q99,kur,skew,lag1_sum,lag1_mean,lag1_std,lag1_min,lag1_max,lag1_q01,lag1_q05,lag1_q25,lag1_q75,lag1_q95,lag1_q99,lag1_kur,lag1_skew,zc_sum,zc_mean,zc_std,zc_q95,zc_kur,zc_skew,mfcc_sum,mfcc_mean,mfcc_std,mfcc_min,mfcc_max,mfcc_q01,mfcc_q05,mfcc_q25,mfcc_q50,mfcc_q75,mfcc_q95,mfcc_q99,mfcc_kur,mfcc_skew,sc_sum,sc_mean,sc_std,sc_min,sc_max,sc_q01,sc_q05,sc_q25,sc_q50,sc_q75,sc_q95,sc_q99,sc_kur,sc_skew,stft_sum,stft_mean,stft_std,stft_min,stft_q01,stft_q05,stft_q25,stft_q50,stft_q75,stft_kur,stft_skew,length_w1,sum_w1,mean_w1,std_w1,min_w1,max_w1,q01_w1,q05_w1,q25_w1,q50_w1,q75_w1,q95_w1,q99_w1,kur_w1,skew_w1,lag1_sum_w1,lag1_mean_w1,lag1_std_w1,lag1_min_w1,lag1_max_w1,lag1_q01_w1,lag1_q05_w1,lag1_q25_w1,lag1_q75_w1,lag1_q95_w1,lag1_q99_w1,lag1_kur_w1,lag1_skew_w1,zc_sum_w1,zc_mean_w1,zc_std_w1,zc_q75_w1,zc_q95_w1,zc_q99_w1,zc_kur_w1,zc_skew_w1,mfcc_sum_w1,mfcc_mean_w1,mfcc_std_w1,mfcc_min_w1,mfcc_max_w1,mfcc_q01_w1,mfcc_q05_w1,mfcc_q25_w1,mfcc_q50_w1,mfcc_q75_w1,mfcc_q95_w1,mfcc_q99_w1,mfcc_kur_w1,mfcc_skew_w1,sc_sum_w1,sc_mean_w1,sc_std_w1,sc_min_w1,sc_max_w1,sc_q01_w1,sc_q05_w1,sc_q25_w1,sc_q50_w1,sc_q75_w1,sc_q95_w1,sc_q99_w1,sc_kur_w1,sc_skew_w1,stft_sum_w1,stft_mean_w1,stft_std_w1,stft_min_w1,stft_max_w1,stft_q01_w1,stft_q05_w1,stft_q25_w1,stft_q50_w1,stft_q75_w1,stft_q95_w1,stft_q99_w1,stft_kur_w1,stft_skew_w1,length_w2,sum_w2,mean_w2,std_w2,min_w2,max_w2,q01_w2,q05_w2,q25_w2,q50_w2,q75_w2,q95_w2,q99_w2,kur_w2,skew_w2,lag1_sum_w2,lag1_mean_w2,lag1_std_w2,lag1_min_w2,lag1_max_w2,lag1_q01_w2,lag1_q05_w2,lag1_q25_w2,lag1_q50_w2,lag1_q75_w2,lag1_q95_w2,lag1_q99_w2,lag1_kur_w2,lag1_skew_w2,zc_sum_w2,zc_mean_w2,zc_std_w2,zc_q75_w2,zc_q95_w2,zc_kur_w2,zc_skew_w2,mfcc_sum_w2,mfcc_mean_w2,mfcc_std_w2,mfcc_min_w2,mfcc_max_w2,mfcc_q01_w2,mfcc_q05_w2,mfcc_q25_w2,mfcc_q50_w2,mfcc_q75_w2,mfcc_q95_w2,mfcc_q99_w2,mfcc_kur_w2,mfcc_skew_w2,sc_sum_w2,sc_mean_w2,sc_std_w2,sc_min_w2,sc_max_w2,sc_q01_w2,sc_q05_w2,sc_q25_w2,sc_q50_w2,sc_q75_w2,sc_q95_w2,sc_q99_w2,sc_kur_w2,sc_skew_w2,stft_sum_w2,stft_mean_w2,stft_std_w2,stft_min_w2,stft_q01_w2,stft_q05_w2,stft_q25_w2,stft_q50_w2,stft_q75_w2,stft_kur_w2,stft_skew_w2,length_w3,sum_w3,mean_w3,std_w3,min_w3,max_w3,q01_w3,q05_w3,q25_w3,q50_w3,q75_w3,q95_w3,q99_w3,kur_w3,skew_w3,lag1_sum_w3,lag1_mean_w3,lag1_std_w3,lag1_min_w3,lag1_max_w3,lag1_q01_w3,lag1_q05_w3,lag1_q25_w3,lag1_q50_w3,lag1_q75_w3,lag1_q95_w3,lag1_q99_w3,lag1_kur_w3,lag1_skew_w3,zc_sum_w3,zc_mean_w3,zc_std_w3,zc_q95_w3,zc_kur_w3,zc_skew_w3,mfcc_sum_w3,mfcc_mean_w3,mfcc_std_w3,mfcc_min_w3,mfcc_max_w3,mfcc_q01_w3,mfcc_q05_w3,mfcc_q25_w3,mfcc_q50_w3,mfcc_q75_w3,mfcc_q95_w3,mfcc_q99_w3,mfcc_kur_w3,mfcc_skew_w3,sc_sum_w3,sc_mean_w3,sc_std_w3,sc_min_w3,sc_max_w3,sc_q01_w3,sc_q05_w3,sc_q25_w3,sc_q50_w3,sc_q75_w3,sc_q95_w3,sc_q99_w3,sc_kur_w3,sc_skew_w3,stft_sum_w3,stft_mean_w3,stft_std_w3,stft_min_w3,stft_q01_w3,stft_q05_w3,stft_q25_w3,stft_q50_w3,stft_q75_w3,stft_kur_w3,stft_skew_w3,length_w4,sum_w4,mean_w4,std_w4,min_w4,max_w4,q01_w4,q05_w4,q25_w4,q50_w4,q75_w4,q95_w4,q99_w4,kur_w4,skew_w4,lag1_sum_w4,lag1_mean_w4,lag1_std_w4,lag1_min_w4,lag1_max_w4,lag1_q01_w4,lag1_q05_w4,lag1_q25_w4,lag1_q75_w4,lag1_q95_w4,lag1_q99_w4,lag1_kur_w4,lag1_skew_w4,zc_sum_w4,zc_mean_w4,zc_std_w4,zc_q75_w4,zc_q95_w4,zc_q99_w4,zc_kur_w4,zc_skew_w4,mfcc_sum_w4,mfcc_mean_w4,mfcc_std_w4,mfcc_min_w4,mfcc_max_w4,mfcc_q01_w4,mfcc_q05_w4,mfcc_q25_w4,mfcc_q50_w4,mfcc_q75_w4,mfcc_q95_w4,mfcc_q99_w4,mfcc_kur_w4,mfcc_skew_w4,sc_sum_w4,sc_mean_w4,sc_std_w4,sc_min_w4,sc_max_w4,sc_q01_w4,sc_q05_w4,sc_q25_w4,sc_q50_w4,sc_q75_w4,sc_q95_w4,sc_q99_w4,sc_kur_w4,sc_skew_w4,stft_sum_w4,stft_mean_w4,stft_std_w4,stft_min_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4,vocal_channel,emotion_angry,emotion_calm,emotion_disgust,emotion_fearful,emotion_happy,emotion_neutral,emotion_sad,emotion_surprised,emotional_intensity_normal,emotional_intensity_strong,statement_Dogs are sitti

### Modello 

In [24]:
X_train = df_train_v.drop(['vocal_channel'], axis=1)
y_train = df_train_v['vocal_channel']
X_test = df_test_v.drop(['vocal_channel'], axis=1)
y_test = df_test_v['vocal_channel']

C_range = np.logspace(-4, 4, 100) # genera 100 valori logaritmici distribuiti uniformemente nell'intervallo [0.001, 1000].
param_grid = {'C': C_range}

clf = LinearSVC(dual=False)
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=100, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) # addestro il modello sul train

y_pred = best_clf.predict(X_test)

print(classification_report(y_test,y_pred))

print("Best C value:", random_search.best_params_['C'])

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       360
           1       0.96      0.99      0.98       264

    accuracy                           0.98       624
   macro avg       0.98      0.98      0.98       624
weighted avg       0.98      0.98      0.98       624

Best C value: 0.08111308307896872


## Variabile target --> Emotional_intensity

### One-hot encoding

In [59]:
df_test_scaled_ei = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [60]:
df_train_scaled_ei = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

#### Train

In [61]:
df_train_scaled_ei['emotional_intensity'] = pd.factorize(df_train_scaled_ei['emotional_intensity'])[0]
df_train_ei = pd.get_dummies(df_train_scaled_ei)

#### Test

In [62]:
df_test_scaled_ei['emotional_intensity'] = pd.factorize(df_test_scaled_ei['emotional_intensity'])[0]
df_test_ei = pd.get_dummies(df_test_scaled_ei)

### Modello

In [64]:
X_train = df_train_s.drop(['emotional_intensity'], axis=1)
y_train = df_train_s['emotional_intensity']
X_test = df_test_s.drop(['emotional_intensity'], axis=1)
y_test = df_test_s['emotional_intensity']

C_range = np.logspace(-4, 4, 100) # genera 100 valori logaritmici distribuiti uniformemente nell'intervallo [0.001, 1000].
param_grid = {'C': C_range}

clf = LinearSVC(dual=False)
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=100, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) # addestro il modello sul train

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred))

print("Best C value:", random_search.best_params_['C'])

              precision    recall  f1-score   support

           0       0.72      0.89      0.80       336
           1       0.83      0.59      0.69       288

    accuracy                           0.75       624
   macro avg       0.77      0.74      0.74       624
weighted avg       0.77      0.75      0.75       624

Best C value: 0.1707352647470692


## Variabile target --> Emotion

### One-hot encoding

In [65]:
df_test_scaled_e = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [66]:
df_train_scaled_e = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

#### Train

In [67]:
df_train_scaled_e['emotion'] = pd.factorize(df_train_scaled_e['emotion'])[0]
df_train_e = pd.get_dummies(df_train_scaled_e)

#### Test

In [68]:
df_test_scaled_e['emotion'] = pd.factorize(df_test_scaled_e['emotion'])[0]
df_test_e = pd.get_dummies(df_test_scaled_e)

### Modello

In [69]:
X_train = df_train_s.drop(['emotion'], axis=1)
y_train = df_train_s['emotion']
X_test = df_test_s.drop(['emotion'], axis=1)
y_test = df_test_s['emotion']

C_range = np.logspace(-4, 4, 100) # genera 100 valori logaritmici distribuiti uniformemente nell'intervallo [0.001, 1000].
param_grid = {'C': C_range}

clf = LinearSVC(dual=False)
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=100, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) # addestro il modello sul train

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred))

print("Best C value:", random_search.best_params_['C'])

              precision    recall  f1-score   support

           0       0.43      0.54      0.48        48
           1       0.43      0.83      0.56        96
           2       0.56      0.15      0.23        96
           3       0.50      0.04      0.08        96
           4       0.61      0.81      0.70        96
           5       0.63      0.56      0.59        96
           6       0.51      0.65      0.57        48
           7       0.46      0.67      0.55        48

    accuracy                           0.51       624
   macro avg       0.52      0.53      0.47       624
weighted avg       0.53      0.51      0.46       624

Best C value: 0.08111308307896872


# Non-Linear SVM

## Variabile target --> Sex

In [70]:
X_train = df_train_s.drop(['sex'], axis=1)
y_train = df_train_s['sex']
X_test = df_test_s.drop(['sex'], axis=1)
y_test = df_test_s['sex']

C_range = np.logspace(-4, 4, 100)
gamma_range = np.logspace(-4, 4, 100) # possibile gamma range
kernel_type = ['rbf', 'poly'] # possibile kernel type
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

              precision    recall  f1-score   support

           0       0.83      0.99      0.91       312
           1       0.99      0.80      0.88       312

    accuracy                           0.90       624
   macro avg       0.91      0.90      0.89       624
weighted avg       0.91      0.90      0.89       624

Best C value: 0.03199267137797385
Best kernel: poly
Best gamma value: 0.03853528593710531
CPU times: total: 8.81 s
Wall time: 8min 34s


In [79]:
C_range = [10,20,30,40,50,60,70,80,90]
gamma_range = [10,20,30,40,50,60,70,80,90] # possibile gamma range
kernel_type = ['poly'] # possibile kernel type
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       312
           1       1.00      0.70      0.82       312

    accuracy                           0.85       624
   macro avg       0.88      0.85      0.85       624
weighted avg       0.88      0.85      0.85       624

Best C value: 10
Best kernel: poly
Best gamma value: 10


## Variabile target --> Vocal_channel

In [71]:
X_train = df_train_v.drop(['vocal_channel'], axis=1)
y_train = df_train_v['vocal_channel']
X_test = df_test_v.drop(['vocal_channel'], axis=1)
y_test = df_test_v['vocal_channel']

C_range = np.logspace(-4, 4, 100)
gamma_range = np.logspace(-4, 4, 100) # possibile gamma range
kernel_type = ['rbf', 'poly'] # possibile kernel type
class_w = ["balanced"]
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type,
               "class_weight": class_w}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

              precision    recall  f1-score   support

           0       0.99      0.96      0.98       360
           1       0.95      0.99      0.97       264

    accuracy                           0.97       624
   macro avg       0.97      0.97      0.97       624
weighted avg       0.97      0.97      0.97       624

Best C value: 2718.5882427329457
Best kernel: poly
Best gamma value: 0.0016297508346206436
CPU times: total: 9.48 s
Wall time: 8min 25s


In [72]:
%%time
C_range = np.logspace(-4, 4, 100)
gamma_range = np.logspace(-4, 4, 100) # possibile gamma range
kernel_type = ['rbf', 'poly'] # possibile kernel type
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       360
           1       0.96      0.99      0.97       264

    accuracy                           0.98       624
   macro avg       0.97      0.98      0.98       624
weighted avg       0.98      0.98      0.98       624

Best C value: 95.45484566618347
Best kernel: rbf
Best gamma value: 0.0006428073117284319
CPU times: total: 8.98 s
Wall time: 7min 55s


In [ ]:
C_range = [50,60,70,80,90,100,110,120,130,140,150,160,170,180]
gamma_range = [10,20,30,40,50,60,70,80,90] # possibile gamma range
kernel_type = ['rbf'] # possibile kernel type
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

## Variabile target --> emotional_intensity

In [73]:
X_train = df_train_s.drop(['emotional_intensity'], axis=1)
y_train = df_train_s['emotional_intensity']
X_test = df_test_s.drop(['emotional_intensity'], axis=1)
y_test = df_test_s['emotional_intensity']

C_range = np.logspace(-4, 4, 100)
gamma_range = np.logspace(-4, 4, 100) # possibile gamma range
kernel_type = ['rbf', 'poly'] # possibile kernel type
class_w = ["balanced"]
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type,
               "class_weight": class_w}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

              precision    recall  f1-score   support

           0       0.85      0.64      0.73       336
           1       0.67      0.87      0.76       288

    accuracy                           0.74       624
   macro avg       0.76      0.75      0.74       624
weighted avg       0.77      0.74      0.74       624

Best C value: 166.81005372000593
Best kernel: rbf
Best gamma value: 0.000774263682681127
CPU times: total: 5.2 s
Wall time: 5min 30s


In [74]:
%%time
C_range = np.logspace(-4, 4, 100)
gamma_range = np.logspace(-4, 4, 100) # possibile gamma range
kernel_type = ['rbf', 'poly'] # possibile kernel type
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

              precision    recall  f1-score   support

           0       0.82      0.72      0.77       336
           1       0.72      0.81      0.76       288

    accuracy                           0.76       624
   macro avg       0.77      0.77      0.76       624
weighted avg       0.77      0.76      0.76       624

Best C value: 166.81005372000593
Best kernel: rbf
Best gamma value: 0.000774263682681127
CPU times: total: 5.19 s
Wall time: 5min 23s


In [ ]:
%%time
C_range = [120,130,140,150,160,170,180,190,200]
gamma_range = [10,20,30,40,50,60,70,80,90] # possibile gamma range
kernel_type = ['rbf'] # possibile kernel type
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

## Variabile target --> Emotion

In [75]:
X_train = df_train_s.drop(['emotion'], axis=1)
y_train = df_train_s['emotion']
X_test = df_test_s.drop(['emotion'], axis=1)
y_test = df_test_s['emotion']

C_range = np.logspace(-4, 4, 100)
gamma_range = np.logspace(-4, 4, 100) # possibile gamma range
kernel_type = ['rbf', 'poly'] # possibile kernel type
class_w = ["balanced"]
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type,
               "class_weight": class_w}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

              precision    recall  f1-score   support

           0       0.41      0.65      0.50        48
           1       0.54      0.71      0.61        96
           2       0.46      0.32      0.38        96
           3       0.57      0.22      0.32        96
           4       0.51      0.89      0.64        96
           5       0.59      0.28      0.38        96
           6       0.49      0.56      0.52        48
           7       0.45      0.46      0.45        48

    accuracy                           0.50       624
   macro avg       0.50      0.51      0.48       624
weighted avg       0.51      0.50      0.47       624

Best C value: 0.018307382802953697
Best kernel: poly
Best gamma value: 0.09770099572992257
CPU times: total: 3.5 s
Wall time: 4min 11s


In [76]:
C_range = np.logspace(-4, 4, 100)
gamma_range = np.logspace(-4, 4, 100) # possibile gamma range
kernel_type = ['rbf', 'poly'] # possibile kernel type
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

              precision    recall  f1-score   support

           0       0.43      0.62      0.51        48
           1       0.51      0.72      0.60        96
           2       0.51      0.36      0.43        96
           3       0.53      0.22      0.31        96
           4       0.49      0.86      0.63        96
           5       0.62      0.27      0.38        96
           6       0.47      0.50      0.48        48
           7       0.44      0.46      0.45        48

    accuracy                           0.50       624
   macro avg       0.50      0.50      0.47       624
weighted avg       0.51      0.50      0.47       624

Best C value: 0.018307382802953697
Best kernel: poly
Best gamma value: 0.09770099572992257


In [82]:
C_range = [10,20,30,40,50,60,70,80,90]
gamma_range = [10,20,30,40,50,60,70,80,90] # possibile gamma range
kernel_type = ["poly"] # possibile kernel type
param_grid = {'C': C_range,
              'gamma': gamma_range,
              'kernel': kernel_type}

clf = SVC()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=200, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ # seleziono il miglior modello trovato dalla random search
best_clf.fit(X_train, y_train) 

y_pred = best_clf.predict(X_test)

print(classification_report(y_test, y_pred)

print("Best C value:", random_search.best_params_['C'])
print("Best kernel:", random_search.best_params_['kernel'])
print("Best gamma value:", random_search.best_params_['gamma'])

              precision    recall  f1-score   support

           0       0.34      0.71      0.46        48
           1       0.58      0.51      0.54        96
           2       0.58      0.23      0.33        96
           3       0.42      0.34      0.38        96
           4       0.43      0.82      0.56        96
           5       0.70      0.27      0.39        96
           6       0.47      0.48      0.47        48
           7       0.43      0.48      0.45        48

    accuracy                           0.46       624
   macro avg       0.49      0.48      0.45       624
weighted avg       0.51      0.46      0.45       624

Best C value: 40
Best kernel: poly
Best gamma value: 40
